#Resources

* https://www.youtube.com/watch?v=ZzgUqFtxgXI
* https://www.youtube.com/watch?v=7aBRk_JP-qY
* https://learn.deeplearning.ai/langchain-chat-with-your-data/lesson/3/document-splitting

#Introduction

Title: Exam MCQ Generator

Game Plan:

*   Divide the document into multiple chuncks and then select chunks at random, pass them to Langchain and ask to make an MCQ out of it

#Installs

In [1]:
!pip install -q python-dotenv

!pip install -q langchain_experimental
!pip install -q langchain

!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requ

In [2]:
!pip install -q pydantic

#Imports

In [3]:
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

#Data Source

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
wizard_in_oz_text_path = '/content/drive/MyDrive/datasets/llm-from-scratch/wizard_in_oz.txt'

In [6]:
with open(wizard_in_oz_text_path, 'r', encoding='utf-8') as f:
  text = f.read()

In [7]:
print(text[:100])

﻿Title: Dorothy and the Wizard in Oz


Author: L. Frank Baum

Illustrator: John R. Neill

Release da


#Env

In [8]:
env_path = '/content/drive/MyDrive/credentials/data-analytics-demo/.env'

In [9]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(env_path)

True

In [11]:
GOOGLE_PALM_API_KEY = os.environ['GOOGLE_PALM_API_KEY']
HUGGINGFACE_API_KEY = os.environ['HUGGINGFACE_API_KEY']
OPEN_AI_API_KEY = os.environ['OPEN_AI_API_KEY']

# GOOGLE_PALM_API_KEY, HUGGINGFACE_API_KEY, OPEN_AI_API_KEY

#Model

In [12]:
langchain_llm = ChatOpenAI(openai_api_key=OPEN_AI_API_KEY, model_name="gpt-3.5-turbo-0613", verbose=False,)

In [43]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [44]:
texts = text_splitter.create_documents([text])

In [45]:
print(f'Total Documents Created: {len(texts)}')

Total Documents Created: 287


In [50]:
prompt = (
    "Text:"
    "\n{text}"
    "\nGenerate an Multiple Choice Question from this text"
    "\nDo not use any outside information"
)

In [51]:
prompt_template = ChatPromptTemplate.from_template(prompt)

prompt_template

ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='Text:\n{text}\nGenerate an Multiple Choice Question from this text\nDo not use any outside information'))])

In [52]:
llm_chain = LLMChain(llm=langchain_llm,prompt=prompt_template,output_key="result")

In [58]:
from langchain.callbacks import get_openai_callback as get_langchain_openai_callback
import random

def get_question():
  doc_index = random.randint(0, len(texts)-1)

  with get_langchain_openai_callback() as cb_langchain:
    response = llm_chain.invoke({"text": texts[doc_index]})

  return {'doc_index': doc_index, 'mcq': response['result'], 'cb_langchain': cb_langchain}

#Testing

In [59]:
response = get_question()

In [64]:
print(response['mcq'])

print('\n')
print(f"doc_index = {response['doc_index']}")
print('\ncosting:\n')
print(f"{response['cb_langchain']}")

What happened to the ground while Dorothy and Zeb were in the buggy?

a) It opened in a wide crack and then closed again
b) It started to sway dangerously from side to side
c) It rose up before them
d) It made a sharp crash and a roar


doc_index = 14

costing:

Tokens Used: 336
	Prompt Tokens: 278
	Completion Tokens: 58
Successful Requests: 1
Total Cost (USD): $0.000533


#Pydantic Demo

In [23]:
from pydantic import BaseModel, EmailStr, validator

class User(BaseModel):
  name: str
  # email: EmailStr
  email: str
  account_id: int

  @validator("account_id")
  def validate_account_id(cls, value):
    if value<=0:
      raise ValueError(f'Account ID cannot be negative: {value}')

    return value

user = User(name='jack', email='a@b.com', account_id=1234)

print(user)
print(user.json())
print(user.dict())
print(user.parse_raw(user.json()))

name='jack' email='a@b.com' account_id=1234
{"name": "jack", "email": "a@b.com", "account_id": 1234}
{'name': 'jack', 'email': 'a@b.com', 'account_id': 1234}
name='jack' email='a@b.com' account_id=1234
